In [0]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

import tensorflow


2.2.0


In [0]:
pip install transformers

In [0]:

import tensorflow
from tensorflow import keras
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam, Nadam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import transformers
from transformers import TFAutoModel, AutoTokenizer
from tqdm.notebook import tqdm
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors

In [0]:
#BUILD MODEL FUNCTION

def build_model(transformer, max_len=512):
  input_word_ids = Input(shape=max_len, dtype=tf.int32, name='input_word_ids')
  sequence_output=transformer(input_word_ids)[0]
  cls_token=sequence_output[:, 0, :]
  out = Dense(1, activation='sigmoid')(cls_token)

  model = Model(inputs=input_word_ids, outputs=out)
  model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC()])

  return model

In [0]:
BATCH_SIZE = 8
MAX_LEN = 512  
MODEL = 'distilbert-base-cased'

In [0]:
transformer_layer = TFAutoModel.from_pretrained(MODEL)
model = build_model(transformer_layer, max_len=MAX_LEN)

In [0]:
# Loads the weights
checkpoint_path = '/content/drive/My Drive/Kaggle Toxic comments competition/models/best_distilbert_model.h5'

model.load_weights(checkpoint_path)

In [0]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 512)]             0         
_________________________________________________________________
tf_distil_bert_model (TFDist ((None, 512, 768),)       65190912  
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 768)]             0         
_________________________________________________________________
dense (Dense)                (None, 1)                 769       
Total params: 65,191,681
Trainable params: 65,191,681
Non-trainable params: 0
_________________________________________________________________


In [0]:
#Function for encoding the word/comment to integer or vector

def regular_encode(texts, tokenizer, maxlen=512):
  enc_di = tokenizer.batch_encode_plus(
      texts,
      return_attention_masks=False,
      return_token_type_ids=False,
      pad_to_max_length=True,
      max_length=maxlen
  )

  return np.array(enc_di['input_ids'])

In [0]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [0]:
AUTO = tf.data.experimental.AUTOTUNE

In [0]:
input_text = 'I love going to church. I find lot of peace there.'

In [0]:
import pandas as pd
dict = {'text': [input_text]}

In [0]:
X_test = pd.DataFrame(data = dict)

In [0]:
X_test.head()

,text
0,I love going to church. I find lot of peace th...


In [0]:
import numpy as np
X_test = regular_encode(X_test.text.values, tokenizer, maxlen=MAX_LEN)

In [0]:
y_test = model.predict(X_test)

In [0]:
y_test

array([[1.457329e-06]], dtype=float32)